<a href="https://colab.research.google.com/github/leokao960811/ProgrammingLang/blob/main/W8_TextAnalyze_Pt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

#實作

In [186]:
import pandas as pd
# read data and put it in a dataframe
# 在 google 工作表載入 gsheets
gsheets2 = gc.open_by_url('https://docs.google.com/spreadsheets/d/1b6WaMdMXtSRzg5PqQ9OXaw-32gAP7W0-yVhrxtsVL1A/edit?usp=sharing').get_worksheet(4)

In [187]:
dicts2 = gc.open_by_url('https://docs.google.com/spreadsheets/d/1b6WaMdMXtSRzg5PqQ9OXaw-32gAP7W0-yVhrxtsVL1A/edit?usp=sharing').get_worksheet(5)
dicts2 = dicts2.get_all_records()
dicts2 = pd.DataFrame(dicts2)

In [188]:
# 讀取所有數據
rows2 = gsheets2.get_all_records()
import pandas as pd
df2 = pd.DataFrame(rows2)

In [115]:
!pip install opencc

In [189]:
from opencc import OpenCC

# 建立簡體轉繁體的轉換器（s2t = 簡體轉繁體）
cc = OpenCC('s2t')

# 對文字欄位轉換簡體 → 繁體
for col in df2.columns:
    if df2[col].dtype == object:
        df2[col] = df2[col].apply(lambda x: cc.convert(x) if isinstance(x, str) else x)

# 顯示轉換後結果（可選）
df2

,date,tag,headline,content
0,2016-01-01,詳細全文,陸軍領導機構火箭軍戰略支援部隊成立大會在京舉行 習近平向中國人民解放軍陸軍火箭軍戰略支援部隊...,中國人民解放軍陸軍領導機構、中國人民解放軍火箭軍、中國人民解放軍戰略支援部隊成立大會2015...
1,2016-01-01,詳細全文,中央軍委印發《關於深化國防和軍隊改革的意見》,經中央軍委主席習近平批准，中央軍委近日印發了《關於深化國防和軍隊改革的意見》。\n《意見》強...
2,2016-01-01,詳細全文,《習近平關於嚴明黨的紀律和規矩論述摘編》出版發行,由中共中央紀律檢查委員會、中共中央文獻研究室編輯的《習近平關於嚴明黨的紀律和規矩論述摘編》一...
3,2016-01-01,詳細全文,以實際行動向黨中央看齊 向高標準努力,廣大黨員幹部正在積極學習習近平總書記在中央政治局專題民主生活會上的重要講話。大家紛紛表示要把...
4,2016-01-01,詳細全文,【年終特稿】關鍵之年 改革挺進深水區,剛剛過去的2015年，是全面深化改革的關鍵之年。改革集中發力在制約經濟社會發展的深層次矛盾，...
...,...,...,...,...
314,2016-01-20,國內,今年經貿摩擦預警點將突破百個,中國貿促會2015年啓動國際經貿摩擦應對工作，目前已經在全國設立了52個經貿摩擦預警點，面向...
315,2016-01-20,國內,陸軍新型作戰力量融入作戰體系運用,以新型作戰力量綜合運用爲重點研究課題的聯合作戰演練，在第31集團軍某地進行。多元感知、防空反...
316,2016-01-20,國際,達沃斯論壇開幕 聚焦“第四次工業革命”,世界經濟論壇2016年年會20號在瑞士小鎮達沃斯開幕，爲期4天。今年年會的主題是“掌控第四次...
317,2016-01-20,國際,IMF報告下調世界經濟增長預期,國際貨幣基金組織19號在倫敦發佈《世界經濟展望報告》，預測2016年和2017年全球經濟將分...


In [190]:
!pip install jieba
!pip install snownlp

In [191]:
import jieba
from snownlp import SnowNLP
import pandas as pd
from collections import Counter
import re
import plotly.express as px

In [192]:
for _, row in dicts2.iterrows():
    word = row['words']
    jieba.add_word(word, freq=10000)
    jieba.suggest_freq(word, tune=True)

In [193]:
# 只保留中文字
def extract_chinese(text):
    return ''.join(re.findall(r'[\u4e00-\u9fa5]', text))

In [194]:
df2['Chinese_Content'] = df2['content'].apply(extract_chinese)

# 使用 Jieba 斷詞
df2['Tokenized'] = df2['Chinese_Content'].apply(lambda x: list(jieba.cut(x, HMM=True)))

# 展平成所有詞語的列表
all_words2 = [word for tokens in df2['Tokenized'] for word in tokens if len(word) > 1]

In [195]:
df2['Tokenized']

,Tokenized
0,"[中國人民解放軍, 陸軍領, 導機構, 中國人民解放軍, 火箭, 軍, 中國人民解放軍, 戰..."
1,"[經, 中央, 軍委, 主席, 習近平, 批准, 中央, 軍委, 近日, 印發, 了關, 於..."
2,"[由, 中共中央, 紀律, 檢查, 委員會, 中共中央, 文獻, 研究室, 編輯, 的, 習..."
3,"[廣大黨員, 幹部, 正在, 積極, 學習, 習近平, 總書記, 在, 中央政治局, 專題,..."
4,"[剛剛, 過去, 的, 年, 是, 全面, 深化改革, 的, 關鍵, 之, 年, 改革, 集..."
...,...
314,"[中國, 貿促, 會, 年, 啓, 動國際, 經貿, 摩擦, 應對, 工作, 目前, 已經,..."
315,"[以, 新型, 作戰, 力量, 綜合, 運用, 爲, 重點, 研究, 課題, 的, 聯, 合..."
316,"[世界, 經濟, 論壇, 年, 年, 會號, 在, 瑞士, 小鎮, 達沃斯, 開幕, 爲, ..."
317,"[國際, 貨幣, 基金, 組織號, 在, 倫敦發, 佈, 世界, 經濟, 展望, 報告, 預..."


In [196]:
import re
import pandas as pd

# 定義角色判定規則的函數
def extract_people(content):
    people = []
    # 特例處理：如果提到特定名稱，直接匹配
    if any(loc in content for loc in ['習近平','習總書記','習主席']):
        people.append('習近平')
    higherups_keywords=r'(常萬全|房峰輝|張陽|趙克石|張又俠|吳勝利|馬曉天|魏鵬遠|劉雲山|呂錫文|王岐山|李克強)'
    match = re.search(higherups_keywords, content)
    if match:
        people.append(match.group(0))
    people_keywords=r'(陳研|高峻馳|韓雯|胡欣|關寶凱|蔡金芝|Cayol|苗家樂|孫瑜|劉先偉|郭海周|尹本熊)'
    match = re.search(people_keywords, content)
    if match:
        people.append(match.group(0))

    return people

# 定義學校提取函數
def extract_organizations(content):
    organizations = []
    # 特例處理：如果提到特定名稱，直接匹配
    if any(loc in content for loc in ['中國人民解放軍','解放軍','習主席']):
        organizations.append('中國人民解放軍')
    if '黨中央' in content:
        organizations.append('中國共產黨')
    if '新華社' in content:
        organizations.append('新華社')
    if '國務院' in content:
        organizations.append('國務院')

    match = re.search(r'(中共中央)', content)
    if match:
        organizations.append(match.group(0))

    return organizations

# 定義角色判定規則的函數
def extract_places(content):
    places = []
    if any(loc in content for loc in ['我國','中國']):
        places.append('中國')
    if '北京' in content:
        places.append('北京')
    inland_keywords = r'(西安|南昌|江蘇|海南|廈門|廣西|四川|河北|浙江|重慶|長春|哈爾濱|廣州|陝西|山東|寧夏|廣西|湖北|河南|甘肅|山西)'
    match = re.search(inland_keywords, content)
    if match:
        places.append(match.group(0))
    country_keywords = r'(日本|韓國|巴黎|澳洲|印尼|巴基斯坦|肯尼亞|敘利亞|朝鮮|印度|俄羅斯|阿富汗|美國|波蘭|德國|伊朗|土耳其|臺灣|布基納法索|緬甸)'
    match = re.search(country_keywords, content)
    if match:
        places.append(match.group(0))


    return places

df2['People'] = df2['Chinese_Content'].apply(lambda x: extract_people(str(x)))
df2['Organizations'] = df2['Chinese_Content'].apply(lambda x: extract_organizations(str(x)))
df2['Places'] = df2['Chinese_Content'].apply(lambda x: extract_places(str(x)))

In [197]:
df2

,date,tag,headline,content,Chinese_Content,Tokenized,People,Organizations,Places
0,2016-01-01,詳細全文,陸軍領導機構火箭軍戰略支援部隊成立大會在京舉行 習近平向中國人民解放軍陸軍火箭軍戰略支援部隊...,中國人民解放軍陸軍領導機構、中國人民解放軍火箭軍、中國人民解放軍戰略支援部隊成立大會2015...,中國人民解放軍陸軍領導機構中國人民解放軍火箭軍中國人民解放軍戰略支援部隊成立大會年月日在八一...,"[中國人民解放軍, 陸軍領, 導機構, 中國人民解放軍, 火箭, 軍, 中國人民解放軍, 戰...","[習近平, 常萬全]","[中國人民解放軍, 中國共產黨, 中共中央]",[中國]
1,2016-01-01,詳細全文,中央軍委印發《關於深化國防和軍隊改革的意見》,經中央軍委主席習近平批准，中央軍委近日印發了《關於深化國防和軍隊改革的意見》。\n《意見》強...,經中央軍委主席習近平批准中央軍委近日印發了關於深化國防和軍隊改革的意見意見強調黨的十八大以來...,"[經, 中央, 軍委, 主席, 習近平, 批准, 中央, 軍委, 近日, 印發, 了關, 於...",[習近平],"[中國人民解放軍, 中國共產黨]",[中國]
2,2016-01-01,詳細全文,《習近平關於嚴明黨的紀律和規矩論述摘編》出版發行,由中共中央紀律檢查委員會、中共中央文獻研究室編輯的《習近平關於嚴明黨的紀律和規矩論述摘編》一...,由中共中央紀律檢查委員會中共中央文獻研究室編輯的習近平關於嚴明黨的紀律和規矩論述摘編一書近日...,"[由, 中共中央, 紀律, 檢查, 委員會, 中共中央, 文獻, 研究室, 編輯, 的, 習...",[習近平],[中共中央],[中國]
3,2016-01-01,詳細全文,以實際行動向黨中央看齊 向高標準努力,廣大黨員幹部正在積極學習習近平總書記在中央政治局專題民主生活會上的重要講話。大家紛紛表示要把...,廣大黨員幹部正在積極學習習近平總書記在中央政治局專題民主生活會上的重要講話大家紛紛表示要把踐...,"[廣大黨員, 幹部, 正在, 積極, 學習, 習近平, 總書記, 在, 中央政治局, 專題,...",[習近平],[中國共產黨],[]
4,2016-01-01,詳細全文,【年終特稿】關鍵之年 改革挺進深水區,剛剛過去的2015年，是全面深化改革的關鍵之年。改革集中發力在制約經濟社會發展的深層次矛盾，...,剛剛過去的年是全面深化改革的關鍵之年改革集中發力在制約經濟社會發展的深層次矛盾集中發力在妨礙...,"[剛剛, 過去, 的, 年, 是, 全面, 深化改革, 的, 關鍵, 之, 年, 改革, 集...",[習近平],[],"[中國, 山東]"
...,...,...,...,...,...,...,...,...,...
314,2016-01-20,國內,今年經貿摩擦預警點將突破百個,中國貿促會2015年啓動國際經貿摩擦應對工作，目前已經在全國設立了52個經貿摩擦預警點，面向...,中國貿促會年啓動國際經貿摩擦應對工作目前已經在全國設立了個經貿摩擦預警點面向萬多家企業發佈了...,"[中國, 貿促, 會, 年, 啓, 動國際, 經貿, 摩擦, 應對, 工作, 目前, 已經,...",[],[],[中國]
315,2016-01-20,國內,陸軍新型作戰力量融入作戰體系運用,以新型作戰力量綜合運用爲重點研究課題的聯合作戰演練，在第31集團軍某地進行。多元感知、防空反...,以新型作戰力量綜合運用爲重點研究課題的聯合作戰演練在第集團軍某地進行多元感知防空反導精確定位...,"[以, 新型, 作戰, 力量, 綜合, 運用, 爲, 重點, 研究, 課題, 的, 聯, 合...",[],[],[]
316,2016-01-20,國際,達沃斯論壇開幕 聚焦“第四次工業革命”,世界經濟論壇2016年年會20號在瑞士小鎮達沃斯開幕，爲期4天。今年年會的主題是“掌控第四次...,世界經濟論壇年年會號在瑞士小鎮達沃斯開幕爲期天今年年會的主題是掌控第四次工業革命與會嘉賓將就...,"[世界, 經濟, 論壇, 年, 年, 會號, 在, 瑞士, 小鎮, 達沃斯, 開幕, 爲, ...",[],[],[]
317,2016-01-20,國際,IMF報告下調世界經濟增長預期,國際貨幣基金組織19號在倫敦發佈《世界經濟展望報告》，預測2016年和2017年全球經濟將分...,國際貨幣基金組織號在倫敦發佈世界經濟展望報告預測年和年全球經濟將分別增長和這兩個數字都比年月...,"[國際, 貨幣, 基金, 組織號, 在, 倫敦發, 佈, 世界, 經濟, 展望, 報告, 預...",[],[],[中國]


In [198]:
from collections import Counter

# Count items in each list within the 'People' column
people_counts = Counter(item for sublist in df2['People'] for item in sublist)
places_counts = Counter(item for sublist in df2['Places'] for item in sublist)
org_counts = Counter(item for sublist in df2['Organizations'] for item in sublist)

# Print the counts
people_counts, places_counts, org_counts

(Counter({'習近平': 85,
          '常萬全': 2,
          '魏鵬遠': 1,
          '陳研': 1,
          '蔡金芝': 2,
          '苗家樂': 1,
          '李克強': 19,
          '劉雲山': 3,
          '孫瑜': 1,
          '呂錫文': 3,
          '王岐山': 1,
          '劉先偉': 1,
          '郭海周': 1,
          '尹本熊': 1}),
 Counter({'中國': 140,
          '山東': 8,
          '北京': 30,
          '哈爾濱': 1,
          '韓國': 7,
          '河北': 5,
          '印尼': 1,
          '海南': 5,
          '巴基斯坦': 6,
          '廈門': 1,
          '肯尼亞': 1,
          '印度': 9,
          '敘利亞': 5,
          '朝鮮': 5,
          '俄羅斯': 7,
          '廣西': 2,
          '浙江': 4,
          '重慶': 4,
          '伊朗': 13,
          '美國': 17,
          '南昌': 2,
          '阿富汗': 2,
          '江蘇': 4,
          '長春': 1,
          '波蘭': 1,
          '德國': 7,
          '湖北': 2,
          '山西': 4,
          '日本': 1,
          '西安': 1,
          '廣州': 2,
          '巴黎': 1,
          '四川': 3,
          '陝西': 3,
          '寧夏': 2,
          '甘肅': 2,
          '緬甸': 2,
   

In [199]:
# 將詞頻轉為 DataFrame 並排序
word_freq_df2 = pd.DataFrame(people_counts.items(), columns=['Word', 'Frequency'])
word_freq_df2 = word_freq_df2.sort_values(by='Frequency', ascending=False).reset_index(drop=True)

# 繪製關鍵字統計的條形圖
fig = px.bar(word_freq_df2.head(30), x='Word', y='Frequency',
             title='人名出現次數統計',
             labels={'Word': '關鍵字', 'Frequency': '出現次數'},
             text='Frequency')

fig.update_layout(
    xaxis_title="關鍵字",
    yaxis_title="出現次數",
    xaxis=dict(showgrid=True),
    yaxis=dict(showgrid=True),
    template='plotly_white'
)

fig.show()

In [200]:
# 將詞頻轉為 DataFrame 並排序
word_freq_df2 = pd.DataFrame(places_counts.items(), columns=['Word', 'Frequency'])
word_freq_df2 = word_freq_df2.sort_values(by='Frequency', ascending=False).reset_index(drop=True)

# 繪製關鍵字統計的條形圖
fig = px.bar(word_freq_df2.head(30), x='Word', y='Frequency',
             title='地名出現次數統計',
             labels={'Word': '關鍵字', 'Frequency': '出現次數'},
             text='Frequency')

fig.update_layout(
    xaxis_title="關鍵字",
    yaxis_title="出現次數",
    xaxis=dict(showgrid=True),
    yaxis=dict(showgrid=True),
    template='plotly_white'
)

fig.show()

In [201]:
# 將詞頻轉為 DataFrame 並排序
word_freq_df2 = pd.DataFrame(org_counts.items(), columns=['Word', 'Frequency'])
word_freq_df2 = word_freq_df2.sort_values(by='Frequency', ascending=False).reset_index(drop=True)

# 繪製關鍵字統計的條形圖
fig = px.bar(word_freq_df2.head(30), x='Word', y='Frequency',
             title='組織名出現次數統計',
             labels={'Word': '關鍵字', 'Frequency': '出現次數'},
             text='Frequency')

fig.update_layout(
    xaxis_title="關鍵字",
    yaxis_title="出現次數",
    xaxis=dict(showgrid=True),
    yaxis=dict(showgrid=True),
    template='plotly_white'
)

fig.show()

In [172]:
from collections import Counter

word_counts2 = Counter(all_words2)
word_counts2

Counter({'中國人民解放軍': 3,
         '陸軍領': 2,
         '導機構': 2,
         '火箭': 21,
         '戰略': 81,
         '支援': 12,
         '部隊': 32,
         '成立': 24,
         '大會': 24,
         '年月日': 16,
         '八一': 2,
         '大樓': 1,
         '隆重': 3,
         '舉行': 30,
         '中共中央': 27,
         '總書': 18,
         '記國家': 12,
         '主席': 88,
         '中央': 246,
         '軍委': 31,
         '習近平': 246,
         '向陸': 1,
         '軍戰略': 6,
         '授予': 5,
         '軍旗': 4,
         '並致': 4,
         '訓詞': 2,
         '代表': 41,
         '黨中央': 97,
         '軍委向': 2,
         '同志': 64,
         '全軍部隊': 2,
         '致以': 5,
         '熱烈': 8,
         '賀強': 1,
         '調要': 4,
         '堅持': 135,
         '以黨': 5,
         '形勢': 44,
         '強軍目標': 5,
         '引領': 28,
         '深入': 64,
         '貫徹': 69,
         '軍事': 21,
         '方針': 18,
         '全面': 263,
         '實施': 64,
         '改革': 220,
         '強軍戰略': 7,
         '堅定': 40,
         '移走': 4,
         '中國': 338,
       

In [173]:
# 將詞頻轉為 DataFrame 並排序
#word_freq_df3 = pd.DataFrame(filtered_word_counts3.items(), columns=['Word', 'Frequency'])
word_freq_df3 = pd.DataFrame(word_counts2.items(), columns=['Word', 'Frequency'])
word_freq_df3 = word_freq_df3.sort_values(by='Frequency', ascending=False).reset_index(drop=True)

# 繪製關鍵字統計的條形圖
fig = px.bar(word_freq_df3.head(30), x='Word', y='Frequency',
             title='關鍵字出現次數統計（前30名）',
             labels={'Word': '關鍵字', 'Frequency': '出現次數'},
             text='Frequency')

fig.update_layout(
    xaxis_title="關鍵字",
    yaxis_title="出現次數",
    xaxis=dict(showgrid=True),
    yaxis=dict(showgrid=True),
    template='plotly_white'
)

fig.show()

#範例


**大家為什麼會來上這門課**
https://github.com/peculab/PythonAI4Beginners/issues/1

In [37]:
import pandas as pd
# read data and put it in a dataframe
# 在 google 工作表載入 gsheets
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1kvaMMSudWLyBV3VWOe09bee9-GQFUxSQWayuVYvVTds/edit?usp=sharing').sheet1

In [38]:
dicts = gc.open_by_url('https://docs.google.com/spreadsheets/d/1kvaMMSudWLyBV3VWOe09bee9-GQFUxSQWayuVYvVTds/edit?usp=sharing').get_worksheet(1)
dicts = dicts.get_all_records()
dicts = pd.DataFrame(dicts)

In [39]:
# 讀取所有數據
rows = gsheets.get_all_records()
import pandas as pd
df = pd.DataFrame(rows)

In [40]:
df.head()

,Account,Date,Cleaned_Content
0,peculab,"Aug 27, 2024",Owner peculab • 學校、姓名、以及你為什麼來上這門課？並留下你的第一個 co...
1,pupupeter,"Sep 2, 2024",Collaborator pupupeter • 高中生們兒早哇，我是這次的助教，我叫家瑋...
2,Junxian0829,"Sep 2, 2024",Collaborator Junxian0829 • 各位好，我是鄭鈞賢，是台師大機電系大...
3,tedlu-tw,"Sep 2, 2024",Collaborator tedlu-tw 各位好，和大家一樣是高中生，就讀北市數位實中，...
4,Eric970308-123,"Sep 2, 2024",Eric970308-123 大家好，我是胡程勛，目前高二，就讀彰化縣溪湖高中207班，在...


In [41]:
!pip install jieba
!pip install snownlp

In [42]:
import jieba
from snownlp import SnowNLP
import pandas as pd
from collections import Counter
import re
import plotly.express as px

In [43]:
for _, row in dicts.iterrows():
    word = row['words']
    jieba.add_word(word, freq=10000)
    jieba.suggest_freq(word, tune=True)

In [44]:
# 只保留中文字
def extract_chinese(text):
    return ''.join(re.findall(r'[\u4e00-\u9fa5]', text))

In [46]:
df['Chinese_Content'] = df['Cleaned_Content'].apply(extract_chinese)

# 使用 Jieba 斷詞
df['Tokenized'] = df['Chinese_Content'].apply(lambda x: list(jieba.cut(x, HMM=True)))

# 展平成所有詞語的列表
all_words = [word for tokens in df['Tokenized'] for word in tokens if len(word) > 1]

In [47]:
df['Tokenized']

,Tokenized
0,"[學校, 姓名, 以及, 你, 為什麼, 來上, 這門課, 並, 留下, 你, 的, 第一,..."
1,"[高中生, 們, 兒, 早, 哇, 我, 是, 這次, 的, 助教, 我, 叫, 家瑋, 現..."
2,"[各位, 好, 我, 是, 鄭鈞賢, 是, 台師大機電系, 大三, 的, 從, 前, 我, ..."
3,"[各位, 好, 和, 大家, 一樣, 是, 高中生, 就, 讀北市, 數位, 實中, 同時,..."
4,"[大家, 好, 我, 是, 胡程勛, 目前, 高二, 就, 讀, 彰化縣, 溪湖高中, 班,..."
...,...
307,"[我, 是, 嘉工, 陳信錩, 希望, 可以, 學到, 東西]"
308,"[我, 是, 南港高中, 高廷翰, 對, 程式, 基礎, 有, 幫助]"
309,"[我, 是, 鳳梨大學, 傳播系, 的, 漁人在網, 路上, 路過, 一起, 來學, 的, ..."
310,"[我, 是, 鳳梨大學, 傳播系, 的, 漁人在網, 路上, 路過, 一起, 來學, 的, ..."


In [48]:
import re
import pandas as pd

# 定義角色判定的助教名單
assistants = ['pupupeter', 'Junxian0829', 'tedlu-tw']  # 根據先前提供的名單

# 定義角色判定規則的函數
def determine_role(row):
    if row['Account'] in assistants:
        return '助教'
    elif '助教' in row['Cleaned_Content'] or 'Collaborator' in row['Cleaned_Content']:
        return '助教'
    elif any(keyword in row['Cleaned_Content'] for keyword in ['老師', '數學老師']):
        return '老師'
    elif re.search(r'(高中|附中|高工|高商|工商)', row['Cleaned_Content']):
        return '學生'
    elif re.search(r'(大學|台師大)', row['Cleaned_Content']):
        return '大學生'
    else:
        return '未知'

# 定義學校提取函數
def extract_school(content):
    # 特例處理：如果提到特定學校，直接匹配
    if '數位實中' in content:
        return '數位實中'
    if '台師大' in content or '臺師大' in content or '師範大學' in content:
        return '臺師大'

    # 正則表達式匹配學校名稱
    school_keywords = r'(高中|附中|高工|高商|工商|大學|中學|國中|國小)'
    match = re.search(r'([\u4e00-\u9fa5]+?' + school_keywords + ')', content)
    if match:
        school = match.group(0)
        # 截取最後四個字作為學校名稱
        return school[-4:]
    return None

# 提取學校名稱
df['School'] = df['Cleaned_Content'].apply(lambda x: extract_school(str(x)))

# 應用函數到數據
df['Role'] = df.apply(determine_role, axis=1)
df['School'] = df['Cleaned_Content'].apply(lambda x: extract_school(str(x)))

In [49]:
df['Role']

,Role
0,未知
1,助教
2,助教
3,助教
4,學生
...,...
307,未知
308,學生
309,大學生
310,助教


In [50]:
df.head()

,Account,Date,Cleaned_Content,Chinese_Content,Tokenized,School,Role
0,peculab,"Aug 27, 2024",Owner peculab • 學校、姓名、以及你為什麼來上這門課？並留下你的第一個 co...,學校姓名以及你為什麼來上這門課並留下你的第一個程式連結的我想要透過寫程式來實現我心中的想法我...,"[學校, 姓名, 以及, 你, 為什麼, 來上, 這門課, 並, 留下, 你, 的, 第一,...",None,未知
1,pupupeter,"Sep 2, 2024",Collaborator pupupeter • 高中生們兒早哇，我是這次的助教，我叫家瑋...,高中生們兒早哇我是這次的助教我叫家瑋現為台灣師範大學歷史系三年級的同學也是個愛大便的小孩子這...,"[高中生, 們, 兒, 早, 哇, 我, 是, 這次, 的, 助教, 我, 叫, 家瑋, 現...",臺師大,助教
2,Junxian0829,"Sep 2, 2024",Collaborator Junxian0829 • 各位好，我是鄭鈞賢，是台師大機電系大...,各位好我是鄭鈞賢是台師大機電系大三的從前我的夢想是成為一名頂尖的歌手可惜因為一些個人因素我唱...,"[各位, 好, 我, 是, 鄭鈞賢, 是, 台師大機電系, 大三, 的, 從, 前, 我, ...",臺師大,助教
3,tedlu-tw,"Sep 2, 2024",Collaborator tedlu-tw 各位好，和大家一樣是高中生，就讀北市數位實中，...,各位好和大家一樣是高中生就讀北市數位實中同時在臺大資工擔任研究助理雖然如此但我總是宣稱自己是...,"[各位, 好, 和, 大家, 一樣, 是, 高中生, 就, 讀北市, 數位, 實中, 同時,...",數位實中,助教
4,Eric970308-123,"Sep 2, 2024",Eric970308-123 大家好，我是胡程勛，目前高二，就讀彰化縣溪湖高中207班，在...,大家好我是胡程勛目前高二就讀彰化縣溪湖高中班在班上擔任的幹部是設備股長下面的程式是我在高一的...,"[大家, 好, 我, 是, 胡程勛, 目前, 高二, 就, 讀, 彰化縣, 溪湖高中, 班,...",溪湖高中,學生


In [ ]:
from collections import Counter

school_counts = Counter(df['School'])

In [ ]:
school_counts

Counter({None: 39,
         '臺師大': 2,
         '數位實中': 2,
         '溪湖高中': 13,
         '中山附中': 25,
         '中山大學': 4,
         '南投高中': 2,
         '羅東高工': 14,
         '鼓山高中': 33,
         '仁武高中': 12,
         '關山工商': 2,
         '東海國中': 1,
         '中山高中': 9,
         '花蓮高工': 7,
         '楠梓高中': 17,
         '中港高中': 6,
         '清水高中': 31,
         '新社高中': 2,
         '林園高中': 15,
         '基隆高中': 30,
         '嘉義高工': 13,
         '南港高中': 19,
         '新竹高商': 10,
         '竹北高中': 2,
         '鳳梨大學': 2})

In [ ]:
import plotly.express as px

# Sorting the school counts in descending order
sorted_school_counts = sorted(school_counts.items(), key=lambda x: x[1], reverse=True)
sorted_school_counts_dict = dict(sorted_school_counts)

# Creating a sorted DataFrame for the bar chart
sorted_school_df = pd.DataFrame(list(sorted_school_counts_dict.items()), columns=["School", "Count"])

# Plotting the sorted school distribution
fig_sorted = px.bar(sorted_school_df, x='School', y='Count',
                    title="School Distribution (Sorted by Count)",
                    labels={"School": "School", "Count": "Number of Mentions"},
                    text='Count')

fig_sorted.update_traces(texttemplate='%{text}', textposition='outside')
fig_sorted.update_layout(xaxis=dict(title="School"), yaxis=dict(title="Number of Mentions"))

fig_sorted.show()

In [ ]:
# 計算每個 Account 的留言次數
account_message_count = df.groupby('Account').size().reset_index(name='Message_Count')

# 合併學校名稱到 account_message_count
account_school = df[['Account', 'School']].drop_duplicates()

# 合併留言次數與學校名稱
result = account_message_count.merge(account_school, on='Account')

In [ ]:
result.describe()

,Message_Count
count,283.000000
mean,1.385159
std,1.565381
min,1.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,11.000000


In [ ]:
result

,Account,Message_Count,School
0,40112,1,鼓山高中
1,40114,1,鼓山高中
2,210136,1,嘉義高工
3,218111,1,羅東高工
4,311257,1,清水高中
...,...,...,...
278,zhen910,1,花蓮高工
279,zhenjane,1,楠梓高中
280,zhenxinglin21024,1,基隆高中
281,zquonn,1,仁武高中


In [ ]:
# Group the data by 'School' and aggregate the message count and accounts
grouped_data = result.groupby('School').agg({
    'Account': lambda accounts: ', '.join(map(str, accounts)),  # Convert integers to strings before joining
    'Message_Count': 'sum'
}).reset_index()

# Rename columns for clarity
grouped_data.rename(columns={
    'School': 'School_Name',
    'Account': 'Accounts',
    'Message_Count': 'Total_Messages'
}, inplace=True)

# Sort the grouped data by Total_Messages in descending order
sorted_data = grouped_data.sort_values(by='Total_Messages', ascending=False).reset_index(drop=True)
sorted_data

,School_Name,Accounts,Total_Messages
0,基隆高中,"21018Hank, An4567ny, Eeeeeeeemma, Kien0516, Le...",40
1,中山附中,"19-bryce, Anson594412, DingSianWu, I2amgay, Ia...",35
2,鼓山高中,"40112, 40114, 131001yt2019snipe, Andy980325, B...",33
3,清水高中,"311257, 311259, 311264, 311272, 10802-a, Allen...",31
4,楠梓高中,"971010ian, CaiBingyan123, Pei-0325, Rebecca-10...",27
5,羅東高工,"218111, 218234nick, JEFF-GMT, Jeff-1024, Jusus...",24
6,南港高中,"05yc, 1024Amon, 10909Jasmine, 10921Hennessy, 1...",19
7,溪湖高中,"Eric970308-123, Jiaxuan210357, Lin434255, Nily...",15
8,林園高中,"000000-Nora, Albert980206, Chuxin014, Irving-1...",15
9,中山大學,"Rna0925, kai081024, peculab",14


In [ ]:
raw_accounts = list(sorted_data[sorted_data['School_Name'] == '東海國中']['Accounts'])

In [ ]:
raw_accounts

['QiuDaru']

In [ ]:
raw_accounts = list(sorted_data[sorted_data['School_Name'] == '鳳梨大學']['Accounts'])

In [ ]:
raw_accounts

['fishpersonn, pupupeter']

In [ ]:
raw_accounts = list(sorted_data[sorted_data['School_Name'] == '基隆高中']['Accounts'])
raw_accounts

['21018Hank, An4567ny, Eeeeeeeemma, Kien0516, Lemon-Huan, Naomi726, RZKlove, RaY-smi1ec0w, Sherrycc-00, Timmmuoo, Xana1001, Yona825, bear97151106, chenliweileo, dilligaf01, dodo-6954, howard09166, jeremy970522, mattr-rrr, peculab, perry0407, t1na-5u, tim970520, tingchen601, txx1n, yongjun924, yuzn3348, zdujulie, zhenxinglin21024']

In [ ]:
accounts_to_extract = set(raw_accounts[0].split(', '))

In [ ]:
accounts_to_extract

{'21018Hank',
 'An4567ny',
 'Eeeeeeeemma',
 'Kien0516',
 'Lemon-Huan',
 'Naomi726',
 'RZKlove',
 'RaY-smi1ec0w',
 'Sherrycc-00',
 'Timmmuoo',
 'Xana1001',
 'Yona825',
 'bear97151106',
 'chenliweileo',
 'dilligaf01',
 'dodo-6954',
 'howard09166',
 'jeremy970522',
 'mattr-rrr',
 'peculab',
 'perry0407',
 't1na-5u',
 'tim970520',
 'tingchen601',
 'txx1n',
 'yongjun924',
 'yuzn3348',
 'zdujulie',
 'zhenxinglin21024'}

In [ ]:
filtered_df = df[df['Account'].isin(accounts_to_extract)]

In [ ]:
filtered_df

,Account,Date,Cleaned_Content,Chinese_Content,Tokenized,School,Role
0,peculab,"Aug 27, 2024",Owner peculab • 學校、姓名、以及你為什麼來上這門課？並留下你的第一個 co...,學校姓名以及你為什麼來上這門課並留下你的第一個程式連結的我想要透過寫程式來實現我心中的想法我...,"[學校, 姓名, 以及, 你, 為什麼, 來上, 這門課, 並, 留下, 你, 的, 第一,...",None,未知
13,peculab,"Sep 2, 2024",Owner Author peculab 大家好，我是王俊剴，目前高一，讀高雄市中山大學附...,大家好我是王俊剴目前高一讀高雄市中山大學附中高一丙班對程式感興趣未來也想往這方面發展,"[大家, 好, 我, 是, 王俊剴, 目前, 高一, 讀, 高雄市, 中山, 大學, 附中,...",中山大學,學生
103,peculab,"Sep 3, 2024",Owner Author peculab 大家好，我是羅東高工電機科的李博堯，希望能對AI...,大家好我是羅東高工電機科的李博堯希望能對有更多的了解所以選擇這門課,"[大家, 好, 我, 是羅東, 高工, 電機科, 的, 李博堯, 希望, 能, 對, 有, ...",羅東高工,學生
138,peculab,"Sep 5, 2024",Owner Author peculab I'm Pecu. My code is htt...,,[],None,未知
139,peculab,"Sep 5, 2024","Owner Author peculab 大家好，我是楊森宇,就讀中山附中高一丙班 想玩玩...",大家好我是楊森宇就讀中山附中高一丙班想玩玩看程式設計,"[大家, 好, 我, 是, 楊森宇, 就, 讀, 中山, 附中, 高一, 丙班想, 玩玩, ...",中山附中,學生
140,peculab,"Sep 5, 2024",Owner Author peculab 我是楠梓高中101黃軍燁 https://col...,我是楠梓高中黃軍燁,"[我, 是, 楠梓, 高中, 黃, 軍燁]",楠梓高中,學生
141,peculab,"Sep 5, 2024",Owner Author peculab 我是明天才要上課的陪同老師~ 今天先預習一下!!...,我是明天才要上課的陪同老師今天先預習一下本身對程式完全基礎的我從網頁到軟體對我來說都是新鮮的...,"[我, 是, 明天, 才, 要, 上, 課, 的, 陪同, 老師, 今天, 先, 預習, 一...",None,老師
165,perry0407,"Sep 5, 2024",perry0407 老師好，我是基隆高中210班的陳品丞，我希望能藉由學習程式感善生活環境...,老師好我是基隆高中班的陳品丞我希望能藉由學習程式感善生活環境,"[老師, 好, 我, 是, 基隆, 高中班, 的, 陳, 品丞, 我, 希望, 能, 藉由,...",基隆高中,老師
178,dilligaf01,"Sep 5, 2024","dilligaf01 大家好我是基隆高中210石兆馨,選這門課是因為覺得程式設計對未來有幫...",大家好我是基隆高中石兆馨選這門課是因為覺得程式設計對未來有幫助,"[大家, 好, 我, 是, 基隆, 高中, 石兆馨, 選, 這門課, 是, 因為, 覺得, ...",基隆高中,學生
179,Lemon-Huan,"Sep 5, 2024",Lemon-Huan 我是國立基隆高中210班的陳禹瑍，希望能夠藉此學習到更多有關AI的知...,我是國立基隆高中班的陳禹瑍希望能夠藉此學習到更多有關的知識並能夠應用在以後生活中的各種方面,"[我, 是, 國立, 基隆, 高中班, 的, 陳, 禹, 瑍, 希望, 能夠, 藉此, 學習...",基隆高中,學生


In [ ]:
from google.colab import userdata
apikey = userdata.get('GOOGLE_API_KEY')

In [ ]:
import google.generativeai as genai
genai.configure(api_key = apikey)
model = genai.GenerativeModel("gemini-1.5-flash")

prompt = f"用中文分析社交媒體上對特定事件的風向：{filtered_df.to_string()}"

response = model.generate_content(prompt)
print(response.text)

這份數據顯示了社交媒體上關於一門程式設計課程的參與者資訊。主要風向可以從以下幾個方面分析：

**1. 參與者構成：**  絕大多數參與者來自基隆高中，僅少數來自其他學校（中山大學附中、羅東高工、楠梓高中）。這表明該課程的參與者主要集中於基隆高中學生，並且可能有老師參與教學或輔導。  

**2. 學習動機：**  基隆高中的學生們普遍表達了學習AI和程式設計的意願，並說明了他們希望藉此提升生活、改善環境，或為未來發展作準備。部分學生提到想讓生活更輕鬆、自由，也有一些學生表達了對程式設計的濃厚興趣，即使覺得困難也會認真學習。  學習動機積極且明確。

**3. 課程內容推測：** 由於學生們提到AI和程式語言學習，可以推測該課程主要圍繞著AI相關的程式設計教學。  Colab連結的頻繁出現也證實了課程可能涉及實際程式編寫和實作。

**4.  peculab帳號的特殊性：** peculab帳號發布了多條信息，內容涵蓋學生介紹、自身程式碼分享以及一些看似與課程無直接相關的目標陳述（例如「想要有五百五十萬台幣年薪」）。這暗示著peculab帳號可能是課程的開設者或主要負責人，其發言可能具有官方性質，但也存在一些個人色彩濃厚的言論。

**5.  整體氛圍：** 社交媒體上的整體氛圍積極向上，學生們表現出對學習的熱情和期待。  基隆高中學生的積極參與，說明該課程在當地獲得了不錯的迴響。


**需要注意的點：**

* 資料中部分學生姓名被隱藏或部分顯示，這可能會影響對參與者背景的完整分析。
*  缺乏負面評價或批評性言論，這可能導致對整體風向的判斷存在偏差。  需要更多資料才能更全面地了解課程的真實情況和社會反響。
*  peculab帳號的發言需要仔細甄別，區分其個人言論和官方資訊。


總而言之，從提供的數據來看，社交媒體上關於這門程式設計課程的風向是積極的，基隆高中的學生參與度高，學習動機明確，課程內容可能與AI和程式語言相關。  但數據量有限，結論的可靠性有待進一步驗證。

